In [1]:
library(PerformanceAnalytics)
library(quantmod)
library(xts) 
library(ggplot2)
library(highcharter)
library(purrr)
library(tidyr)
library(dplyr)

Loading required package: xts
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Attaching package: ‘PerformanceAnalytics’

The following object is masked from ‘package:graphics’:

    legend

Loading required package: TTR
Version 0.4-0 included new data defaults. See ?getSymbols.
Highcharts (www.highcharts.com) is a Highsoft software product which is
not free for commercial and Governmental use

Attaching package: ‘dplyr’

The following objects are masked from ‘package:xts’:

    first, last

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
symbols <- c("AAPL", "DIS", "GE", "JNJ", "KO", "MSFT", "SBUX", "SPY", "TGT")
pr_daily <- getSymbols(symbols, src = "yahoo",
           from = "2007-12-31",
           to = "2020-04-30",
           auto.assign = TRUE) %>% 
  map(~Ad(get(.))) %>% 
  reduce(merge)

‘getSymbols’ currently uses auto.assign=TRUE by default, but will
use auto.assign=FALSE in 0.5-0. You will still be able to use
‘loadSymbols’ to automatically load data. getOption("getSymbols.env")
and getOption("getSymbols.auto.assign") will still be checked for
alternate defaults.

This message is shown once per session and may be disabled by setting 
options("getSymbols.warning4.0"=FALSE). See ?getSymbols for details.

pausing 1 second between requests for more than 5 symbols
pausing 1 second between requests for more than 5 symbols
pausing 1 second between requests for more than 5 symbols
pausing 1 second between requests for more than 5 symbols
pausing 1 second between requests for more than 5 symbols


In [3]:
names(pr_daily) <- symbols
head(pr_daily)

               AAPL      DIS       GE      JNJ       KO     MSFT     SBUX
2007-12-31 24.56307 27.38579 23.19337 46.11504 18.29796 26.80937 8.690571
2008-01-02 24.16129 27.01251 22.99941 45.56885 18.21448 26.52319 8.198091
2008-01-03 24.17245 26.95312 23.02445 45.58267 18.40530 26.63617 7.939115
2008-01-04 22.32725 26.41015 22.54894 45.52044 18.44108 25.89062 7.688627
2008-01-07 22.02839 26.43560 22.63652 46.22566 18.87640 26.06383 7.803257
2008-01-08 21.23599 25.90961 22.14851 46.28097 18.95391 25.19025 8.431596
                SPY      TGT
2007-12-31 113.5825 36.39359
2008-01-02 112.5881 36.03694
2008-01-03 112.5337 36.08060
2008-01-04 109.7759 34.99608
2008-01-07 109.6827 35.65845
2008-01-08 107.9115 35.62934

In [4]:
pr_monthly <- to.monthly(pr_daily, indexAt = "lastof", OHLC = FALSE)
head(pr_monthly)

               AAPL      DIS       GE      JNJ       KO     MSFT     SBUX
2007-12-31 24.56307 27.38579 23.19337 46.11504 18.29796 26.80937 8.690571
2008-01-31 16.78543 25.31574 22.12349 43.65372 17.59133 24.55014 8.028270
2008-02-29 15.50321 27.49608 20.93090 43.12204 17.43032 20.56315 7.633437
2008-03-31 17.79484 26.62225 23.37516 45.14731 18.38383 21.45523 7.429655
2008-04-30 21.57081 27.51305 20.65300 46.69235 17.77979 21.56107 6.890471
2008-05-31 23.40610 28.50566 19.40246 46.77612 17.29354 21.48868 7.722594
                SPY      TGT
2007-12-31 113.5825 36.39359
2008-01-31 106.7151 40.33866
2008-02-29 103.9573 38.39448
2008-03-31 103.0277 36.98598
2008-04-30 107.9383 38.77398
2008-05-31 109.5699 39.04104

In [9]:
rt_monthly <- na.omit(Return.calculate(pr_monthly, method = "log"))
tail(rt_monthly)

                  AAPL         DIS           GE         JNJ          KO
2019-11-30  0.07469382  0.15419468  0.121561349  0.04735075 -0.01166276
2019-12-31  0.09420392 -0.04096141 -0.008902104  0.05916453  0.03586586
2020-01-31  0.05260192 -0.04468126  0.109384643  0.02035761  0.05363929
2020-02-29 -0.12183040 -0.16178340 -0.134794429 -0.09530150 -0.08782116
2020-03-31 -0.07231423 -0.19713541 -0.314020378 -0.02522622 -0.18090758
2020-04-30  0.12354726  0.15014981 -0.187878530  0.13604481  0.06284215
                  MSFT        SBUX           SPY         TGT
2019-11-30  0.05776160  0.01522231  0.0355584020  0.16231877
2019-12-31  0.04090124  0.02872996  0.0286413254  0.02527581
2020-01-31  0.07645594 -0.03577809 -0.0004038932 -0.14648450
2020-02-29 -0.04676473 -0.07379291 -0.0824752371 -0.06678109
2020-03-31 -0.02690008 -0.17649897 -0.1333843154 -0.10245212
2020-04-30  0.11781818  0.15627806  0.1288989223  0.18711444

In [6]:
pw <- rep(1/9, 9)

rt_rebal_mo <- Return.portfolio(rt_monthly, weights = pw, 
                                    rebalance_on = "months")

In [12]:
head(rt_rebal_mo)

           portfolio.returns
2008-01-31       -0.08083828
2008-02-29       -0.04188167
2008-03-31        0.03158003
2008-04-30        0.01390285
2008-05-31        0.01791281
2008-06-30       -0.09355567

In [14]:
#options(repr.plot.width = 1, repr.plot.height = 0.75, repr.plot.res = 100)
plot(rt_rebal_mo$portfolio.returns, main = "Returns") 

ERROR: Error in replayPlot(obj): invalid graphics state


ERROR: Error in replayPlot(obj): invalid graphics state


ERROR: Error in replayPlot(obj): invalid graphics state


In [15]:
rt_rebal_mo <- merge.xts(rt_rebal_mo, rt_monthly)
df <- data.frame(rt_rebal_mo, date = index(rt_rebal_mo))
head(df)

,portfolio.returns,AAPL,DIS,GE,JNJ,KO,MSFT,SBUX,SPY,TGT,date
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<date>
2008-01-31,-0.08083828,-0.38073331,-0.07859813,-0.04722692,-0.054850532,-0.039383857,-0.08803374,-0.07926958,-0.062366178,0.102917717,2008-01-31
2008-02-29,-0.04188167,-0.07946392,0.08261746,-0.05541318,-0.012254197,-0.009194779,-0.17721680,-0.05043086,-0.026182251,-0.049396489,2008-02-29
2008-03-31,0.03158003,0.13786138,-0.03229641,0.11044751,0.045896327,0.053260041,0.04246774,-0.02705878,-0.008982603,-0.037374911,2008-03-31
2008-04-30,0.01390285,0.19243278,0.03291340,-0.12381350,0.033649642,-0.033409064,0.00492061,-0.07533998,0.046561369,0.047210433,2008-04-30
2008-05-31,0.01791281,0.08165552,0.03544198,-0.06246093,0.001792498,-0.027728943,-0.00336281,0.11401088,0.015002940,0.006864178,2008-05-31
2008-06-30,-0.09355567,-0.11979834,-0.07410785,-0.12913069,-0.036623041,-0.083556014,-0.02901842,-0.14466668,-0.087275530,-0.137824431,2008-06-30


In [ ]:
df_narrow <- gather(df, key = "Component", value = "Return", -date)
df_narrow[c(1, 143, 285, 427, 569, 711, 853, 995, 1137, 1279),]

bg_df <- filter(df_narrow, Component == "portfolio.returns") %>% 
  select(-Component)
ggplot(data = filter(df_narrow, Component != "portfolio.returns")) +
  geom_line(data = bg_df, aes(x = date, y = Return), colour = "#484848") + 
  geom_line(alpha = 0.6, aes(x = date, y = Return, group = Component, colour = Component)) + 
  facet_wrap(~Component) + theme_bw() + 
  theme(legend.position = "none") 

In [ ]:
window <- 24

pr_rolling_sd_24 <- rollapply(rt_rebal_mo, FUN = sd,
          width = window) %>% 
  na.omit() 

head(pr_rolling_sd_24)

In [ ]:
highchart(type = "stock") %>% 
  hc_title(text = "24-Month Rolling volatility") %>% 
  hc_add_series(pr_rolling_sd_24) %>% 
  hc_add_theme(hc_theme_flat()) %>% 
  hc_yAxis(
    labels = list(format = "{value}%"), 
    opposite = FALSE) %>%
  hc_navigator(enabled = FALSE) %>% 
  hc_scrollbar(enabled = FALSE) %>% 
  hc_exporting(enabled = TRUE) %>% 
  hc_legend(enabled = FALSE)